# AutoXAI Benchmark를 계산하기

In [1]:
import os; os.chdir("../")
import sys; sys.path.append("lib/AutoXAI")

In [2]:
import numpy as np
import pandas as pd
import pickle
import torch
import xgboost as xgb
import yaml
from models.tab_resnet import TabResNet
from utils import load_dataset, load_model, questions_to_xai_sol, hpo_list
from evaluation_measures import evaluate, linear_scalarization
from hyperparameters_optimization import get_parameters, gp_optimization
from XAI_solutions import get_exp_std, set_up_explainer, get_local_exp


/home/cwl/miniconda3/envs/autoxai/lib/python3.8/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/home/cwl/miniconda3/envs/autoxai/lib/python3.8/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimizat

In [8]:
# 1. 모델과 데이터 불러오기
dataset_nm = "Wine Quality"         # <-- 고정
AUTOXAI_NAME_MAP = {"shap": "SHAP", "lime": "LIME"}
N_SAMPLES = 2048
epochs = 12
bg_size = 50
properties_list = ["robustness", "fidelity", "conciseness"]

# Load data
X_train = np.load(f"data/{dataset_nm}/X_train.npy")
y_train = np.load(f"data/{dataset_nm}/y_train.npy")
X_test = np.load(f"data/{dataset_nm}/X_test.npy")
y_test = np.load(f"data/{dataset_nm}/y_test.npy")
feature_metadata = pickle.load(open(f"data/{dataset_nm}/feature_metadata.pkl", "rb"))
raw_data = pd.read_csv(f"data/{dataset_nm}/raw_data.csv")

class NewTabResNet(TabResNet):
    def predict(self, x):
        x = torch.tensor(x, dtype=torch.float32)
        with torch.no_grad():
            return self.forward(x).max().numpy()
        

for model_nm in ["tab_resnet", "xgb"]:
    for explainer_nm in ["shap", "lime"]:
        print(f"Dataset: {dataset_nm}, Model: {model_nm}, Explainer: {explainer_nm}")

        # Load model
        if model_nm == "xgb":
            model = xgb.XGBClassifier()
            model.load_model(f"data/{dataset_nm}/xgb_model.json")
        elif model_nm == "tab_resnet":
            model = NewTabResNet(X_train.shape[1], 2, num_blocks=1)
            model.load_state_dict(torch.load(f"data/{dataset_nm}/resnet_model.pth"))
            model.eval()

        # 2. AutoXAI context 세팅
        context = {}
        rand_idx = np.random.randint(0, X_test.shape[0], bg_size)
        context["X"] = X_test[rand_idx]
        context["y"] = y_test[rand_idx]
        context["feature_names"] = list(raw_data.columns)
        context["verbose"] = False
        context["task"] = "classification"
        context["question"] = "Why"
        context["session_id"] = f"_{bg_size}_{model_nm}"
        context["scaling"] = "Std"
        context["weights"] = [1, 2, 0.5]
        context["distance"] = "cosine"
        context["explanations"] = []
        context["model"] = model
        context["ES"] = True    # Early stopping 사용
        context["IS"] = True    # Improvement stopping 사용

        # 3. Hyperparameter 최적화 (샘플 수는 고정)
        score_hist = {
            "xai_sol": [], "epoch": [], "aggregated_score": [],
            "parameters": [], "robustness": [], "scaled_robustness": [],
            "fidelity": [], "scaled_fidelity": [],
            "conciseness": [], "scaled_conciseness": []
        }

        autoxai_nm = AUTOXAI_NAME_MAP[explainer_nm]

        # ★ Default parameters로 explainer를 세팅하고 std 계산
        print(f"Preparing default explainer and std for {explainer_nm} ({autoxai_nm})")

        # 1. 기본 parameters 가져오기
        default_parameters = get_parameters(
            autoxai_nm, score_hist, "default", properties_list, context)

        # 2. explainer 초기화
        context['explainer'] = set_up_explainer(autoxai_nm, default_parameters, context)

        # 3. 기본 설명값 계산
        exp_values = np.zeros_like(context["X"])
        for i in range(len(context["X"])):
            exp_values[i] = get_local_exp(autoxai_nm, context["X"][i], default_parameters, context)

        context["explanations"] = exp_values
        print(default_parameters)
        print(exp_values)
        print("Len of explanations:", len(exp_values))

        # # 4. 표준편차 계산
        # all_exp = np.stack([np.array(e) for e in exp_values])
        # stds = np.std(all_exp, axis=0)
        # context[f"{autoxai_nm}_std"] = stds


        # # 3. Hyperparameter Optimization
        # print(f"Start GP optimization for {explainer_nm} ({autoxai_nm})")

        # results = gp_optimization(
        #     autoxai_nm, score_hist, properties_list, context, epochs
        # )

        # # 4. Best Parameter 찾기
        # print("\n---------------------- Best Performing Parameters ----------------------")

        # # 결과를 정리
        # param_list = [r['params'] for r in results]
        # score_list = [r['target'] for r in results]   # 'target' = aggregated_score

        # # 가장 높은 score를 가진 인덱스 찾기
        # best_idx = np.argmax(score_list)
        # best_params = param_list[best_idx]
        # best_score = score_list[best_idx]

        # print(f"Best Parameters for {autoxai_nm}:")
        # print(best_params)
        # print(f"Aggregated Score: {best_score:.6f}")

        # # 결과 저장
        # def clean_params(params):
        #     clean = {}
        #     for k, v in params.items():
        #         if isinstance(v, np.generic):  # np.float64, np.int64 같은 것
        #             clean[k] = v.item()         # Python native 타입으로 변환
        #         else:
        #             clean[k] = v
        #     return clean

        # # 🔥 YAML 저장 전에 깨끗하게 변환
        # best_params_clean = clean_params(best_params)

        # # 저장
        # os.makedirs("results", exist_ok=True)
        # param_save_path = f"results/opt_{dataset_nm.replace(' ','_')}_{model_nm}_{autoxai_nm}.yml"
        # yaml.safe_dump({autoxai_nm: best_params_clean}, open(param_save_path, "w"))

        # print(f"Saved best parameters to {param_save_path}")

        # # 4. 최적화된 파라미터로 설명 생성
        # print(f"Generating explanations with best parameters for {autoxai_nm}")

        # if autoxai_nm == "SHAP":
        #     best_params['nsamples'] = N_SAMPLES
        # else:
        #     best_params['num_samples'] = N_SAMPLES

        # best_params['nfeatures'] = int(best_params['nfeatures'])
        # context['explainer'] = set_up_explainer(autoxai_nm, best_params, context)

        # exp_values = []
        # for i in range(X_test.shape[0]):
        #     exp = np.zeros((X_test.shape[1],))
        #     exp[best_params['most_influent_features']] = get_local_exp(autoxai_nm, X_test[i], best_params, context)
        #     exp_values.append(exp)

        # context["explanations"] = exp_values

        # # 5. 결과 저장
        # with open(f"results/explanations_{dataset_nm.replace(' ','_')}_{model_nm}_{explainer_nm}.pkl", "wb") as f:
        #     pickle.dump(context["explanations"], f)

        # print("Finished.")


Dataset: Wine Quality, Model: tab_resnet, Explainer: shap
Preparing default explainer and std for shap (SHAP)
{'summarize': 'KernelExplainer', 'nsamples': 2048, 'l1_reg': 'auto', 'nfeatures': 11, 'most_influent_features': array([10,  3,  7,  9,  6,  1,  2,  5,  0,  4,  8])}
[-0.03668067966677864, -0.024266099486838468, -0.021196498123351032, -0.019097823776229386, -0.018664671970781293, 0.01788546849108813, -0.01240705800569299, -0.009956012062061804, -0.005720847574889238, -0.0036738137392728493, 0.0009879091142703245, -0.08664727795906685, 0.049829229495096994, 0.047186722542063304, 0.03847734560803476, 0.033753003416295085, -0.028341802684108814, -0.004698590254757712, -0.00267461973111565, -0.00219580771314974, 0.0017111637585004152, -0.0012985964811488707, 0.0722654539408134, 0.04981424623892795, -0.045442882784189065, -0.04411164058278272, -0.03820267785021737, -0.0362859595271381, 0.026646067799492024, 0.012189650896044039, 0.005566859401875258, -0.001086598401273682, -0.0010238

In [11]:
import lime

In [9]:
default_parameters

{'num_samples': 5000,
 'nfeatures': 11,
 'most_influent_features': array([9, 0, 3, 4, 8, 7, 1, 2, 5, 6])}

In [4]:
STOP

NameError: name 'STOP' is not defined

In [ ]:
path = "/data9/cwl/repo/pnpxai/research/benchmark/results/explanations_Wine_Quality_tab_resnet_shap.pkl"
# path = "/data9/cwl/repo/pnpxai/research/benchmark/results/explanations_Wine_Quality_tab_resnet_lime.pkl"
with open(path, "rb") as f:
    explanations = pickle.load(f)

In [ ]:
explanations

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 

In [ ]:
import os; os.chdir("../")

In [ ]:
"""
run_eval.py
-----------
통합 스크립트: 〈코드1〉 산출물 + 〈코드2〉 평가 루틴
"""
import os, yaml, pickle, random, logging
import numpy as np
import pandas as pd
import torch, torch.nn as nn
import xgboost as xgb
from torch.utils.data import DataLoader
from torch.autograd import no_grad

# ---- pnpxai / captum utils (〈코드2〉 일부 그대로 가져옴) -------------
from pnpxai import Experiment, AutoExplanation
from pnpxai.core.modality.modality import Modality
from pnpxai.evaluator.metrics import AbPC, Metric, Complexity, Sensitivity
from pnpxai.explainers.utils.baselines import BaselineFunction
from pnpxai.explainers.base import Tunable
from pnpxai.explainers.types import TunableParameter
from pnpxai.explainers import KernelShap

from captum.attr import (
    Lime, IntegratedGradients, Saliency, InputXGradient, NoiseTunnel, LRP
)
from captum.attr import KernelShap as CaptumKernelShap
from captum.attr._utils.lrp_rules import EpsilonRule
from sklearn.cluster import KMeans as SklearnKMeans
# --------------------------------------------------------------------

# ------------------------- 설정 파트 -------------------------------
DATASET  = "Wine Quality"
MODELS   = ["xgb", "tab_resnet"]           # 〈코드1〉에서 학습했던 모델들
EXPLS    = ["shap", "lime",                # xgb  ────────
            "gradient","gradxinput","ig",  # resnet ──────
            "smoothgrad","lrp"]            

NSAMPLES = 2048
SEED     = 42

# reproducibility
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

# ---------------------- 데이터 로드 함수 ----------------------------
def load_data(dataset_nm):
    X_tr = np.load(f"data/{dataset_nm}/X_train.npy")
    y_tr = np.load(f"data/{dataset_nm}/y_train.npy")
    X_te = np.load(f"data/{dataset_nm}/X_test.npy")
    y_te = np.load(f"data/{dataset_nm}/y_test.npy")
    return X_tr, y_tr, X_te, y_te

# ---------------------- 모델 래퍼 정의 ------------------------------
class TorchModelForXGBoost(nn.Module):
    def __init__(self, xgb_model):
        super().__init__()
        self.xgb_model = xgb_model
        self._dummy_layer = nn.Linear(1, 1)

    def forward(self, x: torch.Tensor):
        if x.ndim >= 3:
            x = x.squeeze(0)
        out = self.xgb_model.predict_proba(x.cpu().numpy())
        return torch.from_numpy(out)
    

def load_models():
    # xgb
    xgb_raw = xgb.XGBClassifier()
    xgb_raw.load_model(f"data/{DATASET}/xgb_model.json")
    mdl_xgb = TorchModelForXGBoost(xgb_raw)

    # tab-resnet
    from models.tab_resnet import TabResNet
    X_tr,_,_,_ = load_data(DATASET)
    mdl_resnet = TabResNet(X_tr.shape[1], 2, num_blocks=1)
    mdl_resnet.load_state_dict(torch.load(f"data/{DATASET}/resnet_model.pth"))
    mdl_resnet.eval()

    return {"xgb": mdl_xgb, "tab_resnet": mdl_resnet}

# --------------------- metric helper (〈코드2〉) ---------------------
class CompoundMetric(Metric):
    def __init__(self, model, cmpd_metrics, weights, **kw):
        super().__init__(model=model, **kw)
        self.cmpd_metrics = cmpd_metrics; self.weights=weights
    def evaluate(self, inputs, targets, attrs):
        vals = torch.zeros(attrs.size(0)).to(attrs.device)
        for w, m in zip(self.weights, self.cmpd_metrics):
            vals += w * m.set_explainer(self.explainer).evaluate(
                    inputs, targets, attrs)
        return vals

# ---------------------- 핵심: 통합 평가 -----------------------------
def evaluate_one(model_key, expl_tag, model, X_te, y_te):
    """(model, expl_tag)에 대해 코드1 출력 파일을 읽어 metric 계산"""
    # 1. 설명값 불러오기 (.pkl or .npy)
    fname_npy = f"results/{DATASET.replace(' ','_')}_{model_key}_{expl_tag}.npy"
    fname_pkl = f"results/explanations_{DATASET.replace(' ','_')}_{model_key}_{expl_tag}.pkl"

    if os.path.isfile(fname_npy):
        exps = np.load(fname_npy)
    elif os.path.isfile(fname_pkl):
        exps = pickle.load(open(fname_pkl,'rb'))
        exps = np.stack(exps)
    else:
        raise FileNotFoundError("Explanation file not found!")

    exps_t = torch.tensor(exps, dtype=torch.float32)
    X_te_t = torch.tensor(X_te, dtype=torch.float32).unsqueeze(1)  # (N,1,F)
    loader = DataLoader(X_te_t, batch_size=32, shuffle=False)

    # 2. pnpxai Experiment 세팅 (모델 종류별 분기)
    modality = Modality(dtype=X_te_t[0].dtype, ndims=2, pooling_dim=1)
    if model_key=="tab_resnet":
        expr = AutoExplanation(
            model=model, data=loader, modality=modality,
            target_input_keys=[0],
            target_class_extractor=lambda out: out.argmax(-1),
            label_key='labels', target_labels=False
        )
    else:  # xgb
        # baseline KMeans util (〈코드2〉)
        class KMeans(BaselineFunction, Tunable):
            def __init__(self, background_data, n_clusters=8):
                self.background_data = background_data
                self.n_clusters = TunableParameter(
                    name='n_clusters',
                    current_value=n_clusters,
                    dtype=int,
                    is_leaf=True,
                    space={'low': 8, 'high': len(background_data)//10, 'step': 10},
                )
                self.kmeans_ = SklearnKMeans(n_clusters).fit(background_data)
                BaselineFunction.__init__(self); Tunable.__init__(self)
                self.register_tunable_params([self.n_clusters])
            def __call__(self, inputs):
                if inputs.ndim==3: inputs=inputs.squeeze(1)
                ids = self.kmeans_.predict(inputs.double().cpu())
                centers = self.kmeans_.cluster_centers_[ids]
                return torch.tensor(centers, dtype=torch.float32).to(inputs.device)

        expr = Experiment(
            model=model, data=loader, modality=modality,
            target_input_keys=[0],
            target_class_extractor=lambda out: out.argmax(-1),
            label_key=-1
        )
        expr.metrics.add('abpc', AbPC)
        expr.explainers.add('kernel_shap', KernelShap)
        expr.modality.util_functions['baseline_fn'].delete('zeros')
        expr.modality.util_functions['baseline_fn'].delete('mean')
        expr.modality.util_functions['baseline_fn'].add('kmeans', KMeans)
        expr.modality.util_functions['baseline_fn'].add_default_kwargs(
            'background_data', X_te)

    # 공통 metric 추가
    expr.metrics.add('cmpx', Complexity)
    expr.metrics.add('sens', Sensitivity)
    expr.metrics.add('cmpd', CompoundMetric)

    # 3. metric 인스턴스 생성
    abpc = expr.create_metric('abpc')      # 필요 시
    cmpx = expr.create_metric('cmpx')
    sens = expr.create_metric('sens')
    cmpd = expr.create_metric(
        'cmpd',
        cmpd_metrics=[abpc, cmpx, sens],
        weights=[.7, -.2, -.1]
    )

    # 4. dummy explainer (KernelShap) – metric계산용
    dummy_exp = expr.create_explainer('kernel_shap')

    # 5. 루프 돌며 평가
    scores = {"abpc":[], "cmpx":[], "sens":[], "cmpd":[]}
    for i in range(len(exps_t)):
        inp = {0: X_te_t[i].unsqueeze(0)}
        tgt = model(X_te_t[i]).argmax(-1)
        attr = exps_t[i].unsqueeze(0)

        scores["abpc"].append(
            abpc.set_explainer(dummy_exp).evaluate(inp, tgt, attr).item())
        scores["cmpx"].append(
            cmpx.set_explainer(dummy_exp).evaluate(inp, tgt, attr).item())
        scores["sens"].append(
            sens.set_explainer(dummy_exp).evaluate(inp, tgt, attr).item())
        scores["cmpd"].append(
            cmpd.set_explainer(dummy_exp).evaluate(inp, tgt, attr).item())

    return {k: np.mean(v) for k,v in scores.items()}

# --------------------------- 실행부 ---------------------------------
if __name__ == "__main__":
    X_tr, y_tr, X_te, y_te = load_data(DATASET)
    models = load_models()

    summary = {}
    for mdl_key in MODELS:
        model = models[mdl_key].to("cpu")
        summary[mdl_key] = {}
        for expl in EXPLS:
            # 파일이 없으면 skip
            f_npy = f"results/{DATASET.replace(' ','_')}_{mdl_key}_{expl}.npy"
            f_pkl = f"results/explanations_{DATASET.replace(' ','_')}_{mdl_key}_{expl}.pkl"
            if not (os.path.isfile(f_npy) or os.path.isfile(f_pkl)):
                continue
            print(f"\n>>> Evaluating  {mdl_key}  /  {expl}")
            res = evaluate_one(mdl_key, expl, model, X_te, y_te)
            summary[mdl_key][expl] = res
            for k,v in res.items():
                print(f"   {k:5s}: {v: .4f}")

    # 결과 요약 저장
    yaml.safe_dump(
        summary,
        open(f"results/eval_summary_{DATASET.replace(' ','_')}.yml","w"))
    print("\n✔︎ All done!  (summary yaml saved)")



>>> Evaluating  xgb  /  shap


/home/cwl/miniconda3/envs/pnpenv/lib/python3.10/site-packages/captum/attr/_core/lime.py:1109: UserWarning: You are providing multiple inputs for Lime / Kernel SHAP attributions. This trains a separate interpretable model for each example, which can be time consuming. It is recommended to compute attributions for one example at a time.
  warnings.warn(
/home/cwl/miniconda3/envs/pnpenv/lib/python3.10/site-packages/captum/attr/_core/lime.py:1109: UserWarning: You are providing multiple inputs for Lime / Kernel SHAP attributions. This trains a separate interpretable model for each example, which can be time consuming. It is recommended to compute attributions for one example at a time.
  warnings.warn(
/home/cwl/miniconda3/envs/pnpenv/lib/python3.10/site-packages/captum/attr/_core/lime.py:1109: UserWarning: You are providing multiple inputs for Lime / Kernel SHAP attributions. This trains a separate interpretable model for each example, which can be time consuming. It is recommended to com

   abpc :  0.0000
   cmpx :  0.0000
   sens :  inf
   cmpd : -inf

>>> Evaluating  xgb  /  lime


/home/cwl/miniconda3/envs/pnpenv/lib/python3.10/site-packages/captum/attr/_core/lime.py:1109: UserWarning: You are providing multiple inputs for Lime / Kernel SHAP attributions. This trains a separate interpretable model for each example, which can be time consuming. It is recommended to compute attributions for one example at a time.
  warnings.warn(
/home/cwl/miniconda3/envs/pnpenv/lib/python3.10/site-packages/captum/attr/_core/lime.py:1109: UserWarning: You are providing multiple inputs for Lime / Kernel SHAP attributions. This trains a separate interpretable model for each example, which can be time consuming. It is recommended to compute attributions for one example at a time.
  warnings.warn(
/home/cwl/miniconda3/envs/pnpenv/lib/python3.10/site-packages/captum/attr/_core/lime.py:1109: UserWarning: You are providing multiple inputs for Lime / Kernel SHAP attributions. This trains a separate interpretable model for each example, which can be time consuming. It is recommended to com

KeyboardInterrupt: 

In [ ]:
model

TorchModelForXGBoost(
  (_dummy_layer): Linear(in_features=1, out_features=1, bias=True)
)